This is the test code for BC agent in the CARLA simulator.

In [1]:
import carla 
import math 
import random 
import time 
import csv
import numpy as np
import cv2
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import array_to_img

2024-12-13 15:58:31.265760: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-13 15:58:31.293348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-13 15:58:31.293380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-13 15:58:31.294146: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 15:58:31.298658: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [3]:
saved_model_path = 'IL_town06_steer_bs_32_resnet_both_all_town.h5' 
loaded_model = tf.keras.models.load_model(saved_model_path)

2024-12-13 15:58:32.736887: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-12-13 15:58:32.736909: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: umd-002872
2024-12-13 15:58:32.736913: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: umd-002872
2024-12-13 15:58:32.736989: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.183.1
2024-12-13 15:58:32.737004: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.183.1
2024-12-13 15:58:32.737006: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.183.1


In [4]:
# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', 2000) 
world = client.get_world()
bp_lib = world.get_blueprint_library()  
spawn_points = world.get_map().get_spawn_points() 
print(spawn_points)

[<carla.libcarla.Transform object at 0x7f85090bbe40>, <carla.libcarla.Transform object at 0x7f85090bbdc0>, <carla.libcarla.Transform object at 0x7f85085a38c0>, <carla.libcarla.Transform object at 0x7f85085a3d40>, <carla.libcarla.Transform object at 0x7f85085a0840>, <carla.libcarla.Transform object at 0x7f85085a3740>, <carla.libcarla.Transform object at 0x7f85085a31c0>, <carla.libcarla.Transform object at 0x7f85085a37c0>, <carla.libcarla.Transform object at 0x7f85085a1fc0>, <carla.libcarla.Transform object at 0x7f85085a19c0>, <carla.libcarla.Transform object at 0x7f85085a02c0>, <carla.libcarla.Transform object at 0x7f85085a3a40>, <carla.libcarla.Transform object at 0x7f85085a3940>, <carla.libcarla.Transform object at 0x7f85085a0b40>, <carla.libcarla.Transform object at 0x7f85085a0fc0>, <carla.libcarla.Transform object at 0x7f85085a1540>, <carla.libcarla.Transform object at 0x7f85085a3140>, <carla.libcarla.Transform object at 0x7f85085a0440>, <carla.libcarla.Transform object at 0x7f85085

In [5]:
settings = world.get_settings()
settings.fixed_delta_seconds = 0.01
world.apply_settings(settings)

760

In [6]:
# Add the ego vehicle
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[425])

# Move the spectator behind the vehicle to view it
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)


In [7]:
print(spawn_points[1])

Transform(Location(x=584.831238, y=-13.577756, z=0.300000), Rotation(pitch=0.000000, yaw=-179.580566, roll=0.000000))


In [8]:
# Set initial camera translation
""""change this part """
bound_x = 0.5 + vehicle.bounding_box.extent.x
bound_y = 0.5 + vehicle.bounding_box.extent.y
bound_z = 0.5 + vehicle.bounding_box.extent.z
#camera_init_trans = carla.Transform(carla.Location(x=+0.8*bound_x, y=+0.0*bound_y, z=1.3*bound_z))
camera_init_trans = carla.Transform(carla.Location(x=1.5, z=2.4), carla.Rotation(pitch=0))  # Front
# Add one of each type of camera
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation') 
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle)


In [9]:
# Define respective callbacks
def rgb_callback(image, data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))


In [10]:
def plot_image(model, image ):

    ## resize the input image 
    image = image[:,:,0:3]
    # Resize the image to 448x448
    image = cv2.resize(image, (448, 448))

    # Define cropping coordinates (x_min, y_min, x_max, y_max)
    left, top, right, bottom = 0, 255, 448, 448

    # Crop the image using array slicing
    cropped_img = image[top:bottom, left:right]
    image = cv2.resize(cropped_img, (160,160))
    image = np.array(image).astype("float32")/255.0
    image = np.reshape(image,(160,160,3))
    image = np.expand_dims(image, axis=0)
    X_train = image
    output= model.predict(X_train)

    return output

In [ ]:
# Initialise parameters and data
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()



sensor_data = {'rgb_image': np.zeros((image_h, image_w, 4)),
               'sem_image': np.zeros((image_h, image_w, 4))}

# OpenCV named window for display
cv2.namedWindow('All cameras', cv2.WINDOW_AUTOSIZE)

# Tile all data in one array
top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image']),axis=1)#, sensor_data['inst_image']), axis=1)

# Display with imshow
cv2.imshow('All cameras',top_row)
cv2.waitKey(1)

# Set sensors recording
camera.listen(lambda image: rgb_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))

# Indefinite while loop
v2 = 0
acc = 0
steering = 0
brake = 0
cnt = 0
class_steer=0

loc = []
with open('location2.csv', 'w', newline='') as file_command:
    writer = csv.writer(file_command,lineterminator='\n',)
    writer.writerow(['X',"Y","Z","pitch","yaw","roll","speed",'waypoint_x','waypoint_y','waypoint_z','width'])
    file_command.close()
while True:
    
    # Tile camera images into one array
    top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image']),axis=1)#, sensor_data['inst_image']), axis=1)
    position = (50, 50)  # (x, y) coordinates

    # Specify the font type
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Specify the font scale (font size)
    font_scale = 0.5

    # Specify the font color in BGR format
    font_color = (0, 0, 0)  # Blue

    # Specify the line type
    line_type = 2

    output_cmd = plot_image(loaded_model, sensor_data['rgb_image'])

    v = vehicle.get_velocity()
    tr = vehicle.get_transform()
    v2= 3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2)
    carla_map = world.get_map()
    waypoint = carla_map.get_waypoint(tr.location, project_to_road=True, lane_type=carla.LaneType.Driving)
    width = waypoint.lane_width

    loc.append([tr.location.x,tr.location.y,tr.location.z,tr.rotation.pitch,tr.rotation.yaw,tr.rotation.roll,v2,waypoint.transform.location.x,waypoint.transform.location.y,waypoint.transform.location.z,width])
    
    print('output_cmd',output_cmd)

    steering =float(output_cmd[0])/4
    class_steer = int(np.argmax(output_cmd[1]))
    class_val = float(np.max(output_cmd[1])*100)

    print('class_steer',class_steer)
    
    # Add the text to the image
    cv2.putText(top_row, 'velocity: '+ f"{v2:.2f}" + '   class percent: ' + f"{class_val:.2f}" + '     steering: '+ f"{steering:.2f}" + '    class steer: '+ str(class_steer), position, font, font_scale, font_color, line_type)
    cv2.imshow('All cameras',top_row)


    if v2<30:
        acc=0.6
    else:
        acc=0
    
    vehicle.apply_control(carla.VehicleControl(throttle=acc, steer=steering))
    
    # Break loop if user presses q
    if cv2.waitKey(1) == ord('q'):
        # opening the csv file in 'w+' mode
        with open('location2.csv', 'a', newline='') as file:
            write = csv.writer(file)
            write.writerows(loc)
        break

# Stop sensors and destroy OpenCV window
camera.stop()
sem_camera.stop()


1/1 [==============================] - 1s 563ms/step
output_cmd [array([[0.05156971]], dtype=float32), array([[8.5946274e-05, 1.6748336e-05, 8.6923393e-07, 2.1462962e-03,
        5.2546678e-05, 1.2357347e-04, 6.5747714e-03, 2.6002509e-04,
        1.5625352e-02, 8.2288253e-01, 1.3492635e-01, 3.2566697e-03,
        5.9958133e-03, 1.6246850e-04, 8.9665846e-04, 6.7543187e-03,
        8.8819826e-05, 3.0238118e-06, 5.4483453e-06, 1.4165754e-04]],
      dtype=float32)]
class_steer 9
1/1 [==============================] - 0s 44ms/step
output_cmd [array([[-0.3946326]], dtype=float32), array([[7.4895687e-04, 3.7665665e-03, 1.9543644e-02, 4.0676046e-02,
        9.9574201e-02, 8.1517482e-01, 1.6284988e-03, 1.5412304e-03,
        1.0340741e-02, 1.1959478e-03, 9.3698438e-04, 2.2002465e-05,
        2.7985058e-03, 7.8222256e-06, 5.9520570e-04, 3.1970244e-06,
        9.5960957e-07, 1.3139375e-03, 1.2945980e-04, 1.3055444e-06]],
      dtype=float32)]
class_steer 5
1/1 [==============================] - 

/tmp/ipykernel_414962/350894379.py:68: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  steering =float(output_cmd[0])/4


1/1 [==============================] - 0s 47ms/step
output_cmd [array([[-0.3616644]], dtype=float32), array([[1.17063202e-04, 2.27464344e-02, 3.56012466e-03, 6.08921237e-03,
        8.41331407e-02, 7.67575145e-01, 4.58821021e-02, 9.16017103e-04,
        3.92279364e-02, 1.66725535e-02, 1.31015573e-03, 4.94410779e-06,
        1.15616005e-02, 5.41056306e-06, 1.70871310e-04, 1.21132621e-06,
        1.36526964e-07, 1.19306351e-05, 1.39153608e-05, 5.66454048e-08]],
      dtype=float32)]
class_steer 5
1/1 [==============================] - 0s 49ms/step
output_cmd [array([[-0.28074718]], dtype=float32), array([[5.6154560e-05, 2.9558631e-02, 5.8642769e-04, 2.2701512e-03,
        5.2983448e-02, 5.0768691e-01, 2.4079627e-01, 6.2143774e-04,
        7.8778371e-02, 6.4358041e-02, 1.6744130e-03, 2.3484672e-06,
        2.0542093e-02, 2.7633494e-06, 7.7018587e-05, 6.4256454e-07,
        8.4274546e-08, 9.4690677e-07, 3.8592088e-06, 1.3615483e-08]],
      dtype=float32)]
class_steer 5
1/1 [==============

: 